In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('mail_data.csv')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization

/Users/dilshantharushika/miniforge3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df.loc[df['Category'] == 'spam', 'Category'] = 1
df.loc[df['Category'] == 'ham', 'Category'] = 0


In [6]:
df['Category'] = df['Category'].astype(int)


In [7]:
X= df['Message']
y= df['Category']

In [8]:
MAX_FEATURES = 200000

In [9]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=1800,
                              output_mode='int')

In [10]:
print(y.dtype)  

int64


In [11]:
vectorizer.adapt(X.values)

In [12]:
vectorized_text = vectorizer(X.values)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)

In [14]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Dense


logistic_model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1800,)),
    tf.keras.layers.Dense(64, activation='relu'),  
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
])


logistic_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0002), 
    loss='binary_crossentropy', 
    metrics=['accuracy']        
)


history = logistic_model.fit(train, validation_data=val, epochs=50)


test_loss, test_accuracy = logistic_model.evaluate(test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



Epoch 1/50
244/244 [==============================] - 1s 1ms/step - loss: 14.3630 - accuracy: 0.7705 - val_loss: 7.7208 - val_accuracy: 0.8243
Epoch 2/50
244/244 [==============================] - 0s 1ms/step - loss: 6.4500 - accuracy: 0.8268 - val_loss: 5.4643 - val_accuracy: 0.8478
Epoch 3/50
244/244 [==============================] - 0s 1ms/step - loss: 4.7286 - accuracy: 0.8414 - val_loss: 2.8086 - val_accuracy: 0.8659
Epoch 4/50
244/244 [==============================] - 0s 1ms/step - loss: 3.6136 - accuracy: 0.8555 - val_loss: 2.3956 - val_accuracy: 0.8850
Epoch 5/50
244/244 [==============================] - 0s 1ms/step - loss: 2.7553 - accuracy: 0.8630 - val_loss: 1.9411 - val_accuracy: 0.8641
Epoch 6/50
244/244 [==============================] - 0s 1ms/step - loss: 2.1804 - accuracy: 0.8809 - val_loss: 1.5457 - val_accuracy: 0.9004
Epoch 7/50
244/244 [==============================] - 0s 1ms/step - loss: 2.1619 - accuracy: 0.8673 - val_loss: 1.3991 - val_accuracy: 0.9067
Epoch